# 1. LOADING

In [1]:
# import packages
import pandas as pd
import graphlab as gl
from sklearn import metrics
from scipy import stats

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(2339529, 54)

In [3]:
# print variable names
#data.columns

# 2. DATA PREPARATION

In [4]:
# adding row index
data["row_index"] = data.index

In [5]:
# converting factors to strings
data["user_id"]       = data.user_id.astype(str)
data["media_id"]      = data.media_id.astype(str)
data["album_id"]      = data.album_id.astype(str)
data["artist_id"]     = data.artist_id.astype(str)
data["genre_id"]      = data.genre_id.astype(str)
data["context_type"]  = data.context_type.astype(str)
data["platform_name"] = data.platform_name.astype(str)
data["user_gender"]   = data.user_gender.astype(str)
data["release_year"]  = data.release_year.astype(str)
data["listen_type"]   = data.listen_type.astype(str)
data["hour_of_day"]   = data.hour_of_day.astype(str)
data["weekday"]       = data.weekday.astype(str)
data["first_flow"]    = data.first_flow.astype(str)

In [6]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# drop skipped songs from training
#tr = tr.query("is_listened == 1")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (2279790, 55)
test: (39821, 55)
known: (2319611, 55)
unknown: (19918, 55)


In [7]:
# data sets with user-media interactions
tr_listen = gl.SFrame(tr[["media_id", "user_id", "is_listened", 
                          "context_type", "platform_name", "first_flow", "hour_of_day", "weekday"]])
ts_listen = gl.SFrame(ts[["media_id", "user_id", "is_listened", 
                          "context_type", "platform_name", "first_flow", "hour_of_day", "weekday"]])
kn_listen = gl.SFrame(kn[["media_id", "user_id", "is_listened", 
                          "context_type", "platform_name", "first_flow", "hour_of_day", "weekday"]])
un_listen = gl.SFrame(un[["media_id", "user_id", "is_listened", 
                          "context_type", "platform_name", "first_flow", "hour_of_day", "weekday"]])

This non-commercial license of GraphLab Create for academic use is assigned to kozodoin@hu-berlin.de and will expire on May 12, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1494954006.log


In [8]:
# additional user-level and media-level data
user_data = data.groupby(["user_id"]).head(1)
song_data = data.groupby(["media_id"]).head(1)

# clearing the memory
data = "Null"
tr   = "Null"
kn   = "Null"

# converting data sets
user_data = gl.SFrame(user_data[["user_id", "user_gender", "user_age", "user_ratio_flow", "user_ratio_full"]])
song_data = gl.SFrame(song_data[["media_id", "genre_id", "artist_id", "release_year", "media_duration"]])

# 3. MODELING

In [9]:
# model parameters
n_factors = 50
m_epochs = 400
regular = 1e-5

## 3.1. TRAINING-VALIDATION

In [10]:
# training the model
model = gl.recommender.factorization_recommender.create(tr_listen, 
                                                        binary_target = True, 
                                                        num_factors = n_factors,
                                                        user_id = "user_id", 
                                                        item_id = "media_id", 
                                                        target  = "is_listened",
                                                        user_data = user_data, 
                                                        item_data = song_data,
                                                        max_iterations = m_epochs,
                                                        regularization = regular)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 2279790 observations with 13250 users and 79311 items.

Data prepared in: 17.431s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.358235

Final training Predictive Error: 0.280999

In [11]:
# forecasting
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict(ts_listen)
pred.head(3)

,row_index,is_listened
2167,2167,0.998757
2168,2168,0.893042
2240,2240,0.656716


In [12]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred["is_listened"])

0.72198134484036924

In [13]:
# saving prediction vector
pred.to_csv(path + "pred_valid/factorization_flow_moredata_50f_400i.csv", index = False)

In [14]:
# clearing the memory
tr_listen = "Null"
ts_listen = "Null"
ts = "Null"

## 3.2. KNOWN-UNKNOWN

In [15]:
# training the model
model = gl.recommender.factorization_recommender.create(kn_listen, 
                                                        binary_target = True, 
                                                        num_factors = n_factors,
                                                        user_id = "user_id", 
                                                        item_id = "media_id", 
                                                        target  = "is_listened",
                                                        user_data = user_data, 
                                                        item_data = song_data,
                                                        max_iterations = m_epochs,
                                                        regularization = regular)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 2319611 observations with 13250 users and 79311 items.

Data prepared in: 17.2822s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.358339

Final training Predictive Error: 0.278796

In [16]:
# forecasting
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict(un_listen)
pred = pred.sort_values("sample_id")
pred.head(3)

,sample_id,is_listened
2332218,0,0.999980
2182792,1,0.342784
2092769,2,0.261024


In [17]:
# saving prediction vector
pred.to_csv(path + "pred_unknown/factorization_flow_moredata_50f_400i.csv", index = False)